Imports 
===========================================================================================================================

In [1]:
# Reload module
import importlib
import utils.general
importlib.reload(utils.general)

# Imports
import os
import numpy as np
import matplotlib.pyplot as plt

# Project imports
from utils.loader import SingleTaskRegressionLoader
from utils.general import get_latest_run_dir, extract_seed_from_dir, extract_timestamp_from_dir, get_all_experiment_runs, get_seed_time_pairs_for_models


Parameters
===========================================================================================================================

In [ ]:
# Get latest run directory
get_latest_run = True
# Models and metadata
model_types = ['IC_FDNet', 'BayesNet', 'GaussHyperNet', 'MLPNet', 'DeepEnsembleNet']
# Seeds
seeds = [0]
# Date-time string
date_times = ["2025-07-11_10-19-43"]
# Check and make sure the seed/ date-time pair is in the list
runs_list = get_all_experiment_runs()
seed_data_time_list = get_seed_time_pairs_for_models(runs=runs_list, model_type_list=model_types)
if seeds is None or date_times is None:
    seed_data_time_list = [(seed, date_time) for seed, date_time in seed_data_time_list]

else:
    seed_data_time_list = [(seed, date_time) for seed, date_time in seed_data_time_list if seed in seeds and date_time in date_times]


Load Data
=

In [33]:
# Pre-allocate dictionaries
loaders = {}
x_train, y_train, x_test, y_test = {}, {}, {}, {}

# Metrics stored as: metrics[metric_name][(seed, date_time)][model] = value
metrics = {
    "mean": {}, "var": {}, "std": {}, "res_precision": {}, "res_accuracy": {},
    "bias": {}, "mse": {}, "bias_var_diff": {}, "nll": {}
}

# Losses, mse, and kl stored as: losses[loss_metric][(seed, date_time)][model] = value
losses = {
    "losses": {}, "mses": {}, "kls": {}, "beta": {}
}


for seed_date_time in seed_data_time_list:
    seed, date_time = seed_date_time
    for model in model_types:
        try:
            # Load data
            loader = SingleTaskRegressionLoader(model, seed, date_time)
            metric_dict = loader.load_metrics()
            losses_dict = loader.load_loss_curve()

            # Initialize nested dicts for loader
            if (seed, date_time) not in loaders:
                loaders[(seed, date_time)] = {}
            loaders[(seed, date_time)][model] = loader

            # Initialize nested dicts for metrics
            for k in metrics:
                if (seed, date_time) not in metrics[k]:
                    metrics[k][(seed, date_time)] = {}
                metrics[k][(seed, date_time)][model] = metric_dict[k]

            # Initializes nested dicts for losses
            for k in losses:
                if (seed, date_time) not in losses[k]:
                    losses[k][(seed, date_time)] = {}
                losses[k][(seed, date_time)][model] = losses_dict[k]

        except FileNotFoundError:
            print(f"Skipping missing result: {model}_seed{seed}_{date_time}")

        finally:
            # Log attempt
            print(f"Checked: {model}_seed{seed}_{date_time}")

    # Load shared data once per (seed, date_time)
    data = loader.load_data()
    x_train[seed_date_time] = data["x_train"]
    y_train[seed_date_time] = data["y_train"]
    x_test[seed_date_time] = data["x_test"]
    y_test[seed_date_time] = data["y_test"]

run_names = list(loaders.keys())
print("Loaded runs:", run_names)


Checked: IC_FDNet_seed0_2025-07-11_10-19-43


KeyError: 'beta'

Load Data
=

In [26]:
# Load training and test data
x_train, y_train, x_test, y_test = {}, {}, {}, {}
for run in seed_data_time_list:
    data = loaders[run].load_data()
    x_train[run] = data["x_train"]
    y_train[run] = data["y_train"]
    x_test[run] = data["x_test"]
    y_test[run] = data["y_test"]
print(data.keys())

AttributeError: 'dict' object has no attribute 'load_data'